In [1]:
# Import packages
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor
from IPython.display import clear_output

# Set working path
path = '/Users/martinbogaert/Desktop/NBA Data Analysis/nba_ai_awards/'

print('MAKE SURE YOU CHANGE THE WEEK VARIABLE')

MAKE SURE YOU CHANGE THE WEEK VARIABLE


In [2]:
week = 3 # CHANGE THIS
date = datetime.now().strftime('%d %B %Y')
time = datetime.now().strftime('%H:%M:%S')
print(date + ' ; ' + time)

07 November 2022 ; 10:04:52


In [3]:
features = ['PTS','VORP','FG','WS','MP','TRB','AST','PER','BPM']#, 'GS'] ### GS
model = RandomForestRegressor(n_estimators = 100, max_features = 'auto', min_samples_leaf = 2)

# Minimum minutes per game
mp = 8
# Propotion of maximum games played
gp = 1/4

In [4]:
# Load training data
train_data = pd.read_csv(path + 'Algorithm/roy/roy_data.csv')

# Load predicted data
data = pd.read_csv(path + f'Algorithm/weekly data/week_{week}.csv') # Load up-to-date data
data = data.groupby('roy').get_group(1)

# Load weekly info
with open(path + f'Algorithm/weekly data/week_{week}.txt', 'r') as file:
            date1 = file.readline().splitlines()[0][7:]
            date2 = file.readline().splitlines()[0][7:]
            n = int(file.readline()[12:])
            
season_progress = n / 15
data.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A,roy,smoy
9,Ron Harper Jr.,SF,22,TOR,1,0,3.0,0.0,1.0,0.000,...,7.3,116.84,109.98,6.86,9.00,117.62,108.99,8.63,1,1
11,Christian Koloko,C,22,TOR,10,5,17.4,1.5,3.2,0.469,...,7.3,116.84,109.98,6.86,9.00,117.62,108.99,8.63,1,1
21,Kennedy Chandler,PG,20,MEM,5,0,9.2,0.8,2.8,0.286,...,1.9,116.45,114.55,1.90,1.74,116.31,114.61,1.70,1,1
25,Jake LaRavia,PF,21,MEM,7,0,18.0,2.1,4.3,0.500,...,1.9,116.45,114.55,1.90,1.74,116.31,114.61,1.70,1,1
26,Kenneth Lofton Jr.,PF,20,MEM,3,0,4.7,1.3,3.0,0.444,...,1.9,116.45,114.55,1.90,1.74,116.31,114.61,1.70,1,1


In [5]:
# Set training data
X_train = train_data[features]
y_train = train_data['Share']

# Set minimum requirements (games and minutes played)
data = data[data['MP'] >= mp] # Minimum of minutes per game
data = data[data['G'] > season_progress * gp] # Minimum games played

pred = []
for step in range(0, 10) :
    print('Model ' + str(step+1) + '/10 ...')
    clear_output(wait = True)
    # Fit Machine Learning model
    model.fit(X_train, y_train)

    # Predict shares of test data
    pred.append(model.predict(data[features]))

Model 10/10 ...


In [6]:
# Assemble results DataFrame
res = data.assign(Share = [np.array(pred)[:,i].mean() for i in range(0, len(data))])
res = res.sort_values('Share', ascending = False) # Sort values by predictions
res['Rank'] = list(range(1, len(res)+1))
res['week'] = len(res) * [week] # Add week

res = res.reset_index(drop = True)[['Player', 'MP' ,'Tm', 'PER', 'VORP', 'WS', 'FG', 'PTS', 'Seed', 'Share']]
res.head(5)

,Player,MP,Tm,PER,VORP,WS,FG,PTS,Seed,Share
0,Paolo Banchero,34.6,ORL,18.8,1.640000,4.920000,8.1,22.9,14,0.528899
1,Bennedict Mathurin,28.1,IND,18.4,0.911111,5.466667,6.3,20.7,7,0.368698
2,Dyson Daniels,11.8,NOP,19.9,2.050000,4.100000,1.8,4.8,7,0.065156
3,Jaden Ivey,31.6,DET,12.6,-0.911111,0.911111,5.6,15.0,14,0.055802
4,Walker Kessler,14.3,UTA,20.1,1.025000,5.125000,2.0,5.1,2,0.051474


In [8]:
res.to_csv(path + f'Results/roy/results_week_{week}.csv', index = None)